# **Urban Heat Island (UHI) Assessment in Lahore Using Landsat-8 Thermal Imagery**

1. Install and Import Dependencies

In [1]:
!pip install earthengine-api geemap matplotlib seaborn pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.0 MB/s eta 0:00:00


In [2]:
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics import r2_score
import time
import json
import sys

2. Authenticate & Initialize Earth Engine

In [3]:
ee.Authenticate()
ee.Initialize(project='gee-lahore-lulc')

3. Define ROI-Lahore bounding polygon

In [4]:
lahore_roi=ee.Geometry.Polygon([
    [[74.25, 31.70],
     [74.25, 31.35],
     [74.60, 31.35],
     [74.60, 31.70],
     [74.25, 31.70]
     ]
                                ])

centroid = lahore_roi.centroid().coordinates().getInfo()[::-1]

4. Function to mask clouds

In [5]:
def masks_clouds_l8(img):
    # Check if the image has a QA_PIXEL band
    band_names = img.bandNames()
    condition = band_names.contains('QA_PIXEL')

    def apply_mask(img):
        qa = img.select('QA_PIXEL')
        cloud_mask = qa.bitwiseAnd(1 << 3).eq(0)  # Cloud bit
        shadow_mask = qa.bitwiseAnd(1 << 4).eq(0)  # Shadow bit
        mask = cloud_mask.And(shadow_mask)
        return img.updateMask(mask)

    # Apply mask only if QA_PIXEL exists
    return ee.Image(ee.Algorithms.If(condition, apply_mask(img), img))


5. Create a Landsat-8 collection function

In [6]:
START='2024-05-01'
END='2024-09-30'

sr_coll=(ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
         .filterBounds(lahore_roi)
         .filterDate(START,END)
         .filter(ee.Filter.lt('CLOUD_COVER',30))
         )
toa_coll=(ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')
         .filterBounds(lahore_roi)
         .filterDate(START,END)
         .filter(ee.Filter.lt('CLOUD_COVER',20))
)

def join_sr_to_toa(sr,toa):
  return sr.addBands(toa.select('B10'),overwrite=True)

joined=ee.Join.inner('system:index','system:index').apply(sr_coll,toa_coll,ee.Filter.equals('system:index','system:index'))

def merge_join(f):
  sr=ee.Image(f.get('primary'))
  toa=ee.Image(f.get('secondary'))
  return sr.addBands(toa.select('B10'))

merged_coll=ee.ImageCollection(joined.map(merge_join))
try:
  size=merged_coll.size().getInfo()
  if size == 0:
    merged_coll=sr_coll
except Exception:
  merged_coll=sr_coll

merged_coll=merged_coll.map(masks_clouds_l8)
composite=merged_coll.median().clip(lahore_roi)



6. Compute NDVI and NDBI on composite

In [7]:
bands_present=composite.bandNames().getInfo()
print("Composite bands: ",bands_present)

RED='SR_B4'
NIR ='SR_B5'
SWIR='SR_B6'

for b in [RED,NIR,SWIR]:
  if b not in bands_present:
    raise RuntimeError(f"Expected band {b} in composite bur found only {bands_present}")

ndvi=composite.normalizedDifference([NIR,RED]).rename('NDVI')
ndbi=composite.normalizedDifference([NIR,SWIR]).rename('NDBI')

composite=composite.addBands(ndvi).addBands(ndbi)

Composite bands:  ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'SR_QA_AEROSOL', 'ST_B10', 'ST_ATRAN', 'ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_QA', 'ST_TRAD', 'ST_URAD', 'QA_PIXEL', 'QA_RADSAT']


7. Compute Land Surface Temprature

In [15]:
lambda_val = 10.895e-6
rho = 1.438e-2

def calc_lst(img):
    # Make sure thermal band exists (ST_B10)
    has_b10 = img.bandNames().contains('ST_B10')

    def compute():
        b10 = img.select('ST_B10')

        ndvi_img = img.select('NDVI')
        ndvi_soil = 0.2
        ndvi_veg = 0.5
        pv = ndvi_img.subtract(ndvi_soil).divide(ndvi_veg - ndvi_soil).clamp(0, 1)
        pv = pv.pow(2).rename('Pv')

        emissivity = pv.multiply(0.004).add(0.986).rename('Emissivity')

        # Convert surface temperature (already in Kelvin * 0.00341802 scaling factor)
        lst = b10.multiply(0.00341802).subtract(273.15)  # Convert to °C
        lst = lst.rename('LST')

        return img.addBands([pv, emissivity, lst])

    return ee.Image(ee.Algorithms.If(has_b10, compute(), img))


# -------------------------------------------------------------------
# Apply function to each image
# -------------------------------------------------------------------
def compute_lst_per_image(img):
    img = img.addBands(img.normalizedDifference([NIR, RED]).rename('NDVI'))
    img = img.addBands(img.normalizedDifference([NIR, SWIR]).rename('NDBI'))
    return calc_lst(img)

lst_collection = merged_coll.map(compute_lst_per_image)
lst_median = lst_collection.select('LST').median().clip(lahore_roi)

composite = composite.addBands(lst_median.rename('LST'))

8. Mapping

In [17]:
ndvi_vis = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
ndbi_vis = {'min': -1, 'max': 1, 'palette': ['white', 'orange', 'brown']}
lst_vis = {'min': 290, 'max': 320, 'palette': ['blue', 'cyan', 'green', 'yellow', 'red']}

Map = geemap.Map(center=centroid, zoom=10)
Map.addLayer(ndvi, ndvi_vis, 'NDVI')
Map.addLayer(ndbi, ndbi_vis, 'NDBI')
Map.addLayer(lst_median, lst_vis, 'LST (°C)')
Map.addLayer(lahore_roi, {}, 'Lahore ROI')

# Fixed line
Map.add_colorbar_branca(colors=lst_vis['palette'], vmin=lst_vis['min'], vmax=lst_vis['max'], label="Land Surface Temperature (°C)")

Map


Map(center=[31.52500982410353, 74.42499999999899], controls=(WidgetControl(options=['position', 'transparent_b…